In [1]:
import matplotlib.pyplot as plt
import json
from gresit.simulation_utils import draw_result_boxplots
from collections import defaultdict
import os
import re

2025-05-20 10:54:27,959 - /home/gok4bh/Projects/causal-HU10/venv_causal-HU10/lib/python3.10/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2025-05-20 10:54:28,112 - /home/gok4bh/Projects/causal-HU10/venv_causal-HU10/lib/python3.10/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.
/home/gok4bh/Projects/causal-HU10/venv_causal-HU10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SMALL_SIZE = 15
BIGGER_SIZE = 19

plt.rc("font", size=SMALL_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=BIGGER_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)

In [3]:
result_path = "./"

In [4]:
pc_path = "./PC/"

In [5]:
result_files = [f for f in os.listdir(result_path) if os.path.isfile(os.path.join(result_path, f))]

In [6]:
pc_files = [
    f for f in os.listdir(pc_path) if os.path.isfile(os.path.join(pc_path, f)) and "_PC_" in f
]

In [7]:
gp_files = [result_file for result_file in result_files if "GP" in result_file]

In [8]:
pattern = r"n_(\d+)_gs_(\d+)_snr=\d+_equ_([A-Za-z]+)_(\d+)"
new_pattern = r"n_(\d+)_gs_(\d+)_snr_\d+_equ_([A-Za-z]+)_(\d+)"
short_names = [
    "GRESIT-MURGS",
    "GRESIT-IND",
    "GGraN-DAG",
    "GRandReg",
    "GPC",
]

short_names_no_pc = [
    "GRESIT-MURGS",
    "GRESIT-IND",
    "GGraN-DAG",
    "GRandReg",
]

In [9]:
def _reorder_dict(d, order):
    new_d = {}
    # First, add keys in the specified order if they exist in d.
    for key in order:
        if key in d:
            new_d[key] = d[key]
    # Then add any remaining keys from d (in their original order).
    for key in d.keys():
        if key not in new_d:
            new_d[key] = d[key]
    return new_d

In [10]:
intended_order = [
    "GRESIT-MURGS",
    "GRESIT-IND",
    "GPC",
    "GGraN-DAG",
    "GRandReg",
]

In [ ]:
plot_dict = {}

for result in gp_files:
    with open(result_path + result) as f:
        results: defaultdict[str, defaultdict[str, list[int | float | None]]] = json.load(f)

    pc_result_file = result.replace("results", "results_PC", 1)
    try:
        with open(pc_path + pc_result_file) as pc_f:
            pc_results: defaultdict[str, defaultdict[str, list[int | float | None]]] = json.load(
                pc_f
            )
    except FileNotFoundError:
        continue
    pc_key = [key for key in results.keys() if "GroupPC" in key]
    if pc_key:
        results.pop(pc_key[0])
    results["GroupPC(alpha=0.2, test=FisherZVec)"] = pc_results[
        "GroupPC(alpha=0.2, test=FisherZVec)"
    ]

    results_with_long_names = results
    if len(results_with_long_names) == len(short_names):
        with_short_names = {
            old_name: short_names[i] for i, old_name in enumerate(results_with_long_names.keys())
        }
    elif len(results_with_long_names) == len(short_names_no_pc):
        with_short_names = {
            old_name: short_names_no_pc[i]
            for i, old_name in enumerate(results_with_long_names.keys())
        }
    else:
        raise ValueError()

    results_renamed = {
        with_short_names[key]: value for key, value in results_with_long_names.items()
    }
    # Apply regex search
    match = re.search(pattern, result)
    match_new = re.search(new_pattern, result)

    if match:
        nodes = int(match.group(1))  # Extract number of nodes
        group_size = int(match.group(2))  # Extract group size
        nonlinearity = match.group(3)  # Extract nonlinearity (e.g., "GP")
        sample_size = int(match.group(4))  # Extract sample size
    elif match_new:
        nodes = int(match_new.group(1))  # Extract number of nodes
        group_size = int(match_new.group(2))  # Extract group size
        nonlinearity = match_new.group(3)  # Extract nonlinearity (e.g., "GP")
        sample_size = int(match_new.group(4))  # Extract sample size
    else:
        raise ValueError()

    final_dict = _reorder_dict(results_renamed, intended_order)
    plot_dict[f"result_boxplot_{nodes}_{group_size}_{sample_size}"] = final_dict

    draw_result_boxplots(
        result_dict=final_dict,
        file_path=None,  # Replace with your desired file path
        file_name=f"result_boxplot_{nodes}_{group_size}_{sample_size}",
        title=f"$p={nodes}$, $d_j={group_size}$, and $n={sample_size}$",
    )